In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics.nn.modules.block import C2f
from ultralytics import YOLO


class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // reduction, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels // reduction, in_channels, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(x_cat))

class CBAM(nn.Module):
    def __init__(self, channels, reduction=16):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(channels, reduction)
        self.sa = SpatialAttention()

    def forward(self, x):
        x = x * self.ca(x)
        x = x * self.sa(x)
        return x


In [ ]:
class C2fCBAM(nn.Module):
    def __init__(self, original_c2f):
        super(C2fCBAM, self).__init__()
        self.c2f = original_c2f
        out_channels = original_c2f.cv2.conv.out_channels  
        self.cbam = CBAM(out_channels)

    def forward(self, x):
        x = self.c2f(x)
        return self.cbam(x)

In [ ]:
def inject_cbam_blocks(model):
    for i, layer in enumerate(model.model):
        if isinstance(layer, C2f) and i >= 5:  
            model.model[i] = C2fCBAM(layer)
    return model

In [ ]:
def train_model():
    model = YOLO("yolov8n.pt")
    model.model = inject_cbam_blocks(model.model)

    model.train(
        data="/home/zaman/Code/Breast_Cancer_Detection_and_Monitoring/Cancer.v1i.yolov8/data.yaml",
        epochs=100,
        imgsz=640,
        batch=4,
        optimizer="AdamW",
        lr0=0.0005,
        weight_decay=0.0005,
        amp=True,
        device=0,
        save=True
    )

    return model

trained_model = train_model()
